In [2]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text


from bs4 import BeautifulSoup

import nltk, string, contractions, random, statistics
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [3]:
df = pd.read_csv('data/model_ready_data')

In [4]:
df

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,model_ready
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,door close know door closed turn irritating ha...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man cook evenly fast bit room counter room spa...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,surprise large microwav
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,ordered received ago
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,door close know door closed turn google proble...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,33705501,R34GY91QSWWUE4,B0009KMYGY,4.0,60.0,64.0,N,Just right for me...,The Panasonic NN-H765WF was exactly what I was...,2006-03-23,74,The Panasonic NN-H765WF was exactly what I was...,h765wf exactly looking wanted white match appl...
1871,39268146,R7N1LRCV8GU6L,B0009KMYHI,5.0,5.0,10.0,Y,good looking-good cooking,this microwave cooks food very evenly-no need ...,2006-03-19,21,this microwave cooks food very evenly-no need ...,cook food evenly need stir think best looking ...
1872,19314267,R15KWS9QSI7URC,B0009KMYDM,5.0,12.0,13.0,Y,Panasonic NN-H965WF Luxury full-size 2.2 cu.ft...,"its a great microwave , i use it every day , i...",2006-03-16,67,"its a great microwave , i use it every day , i...",large fit heat evenly unlike defrosting fast m...
1873,11400155,RK5HSUM1GNRH8,B0009KMYDM,5.0,4.0,9.0,N,Panasonic Microwave Oven NN-965WF,Have had it for only a short time but it has p...,2006-02-24,20,Have had it for only a short time but it has p...,short time performed flawlessly purchas


In [5]:
df.dropna(inplace=True)
df.review_date = pd.to_datetime(df.review_date)

In [7]:
keep_words = ['not', 'fire', 'off']
test_stop_words = []
review_stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
              'mom', 'christmas', 'gift', 'got', 'way', 'le', 'daughter', "i'mwork",
              'e','not','love','good','bought','great microwave','micro','nicely',
              'great oven','microwave','product','work great','nice work', "i'm",
              'work great use','work great love','feature work great', 'genius',
              'unit work great','oven work great','easy use love','old oven',
              'old old','20 year old','unit', 'not', 'work', 'amazon', 'com', 'cu',
              'old', 'wife', 'highly', 'recommend', 'like', 'charm', 'sharp', 'ft', 
              'easy', 'oven', 'use', 'year', 'lot', 'pleased', 'happy', 'hope',
              'review', 'buy', 'far', 'day', '1', '2', '3', '4', '5', '6', '7',
              '8', '9', '10', '20', 'great', 'excellent','feature', 'nice', 'old',
              'geniusunit', "don't", "doesn't", '34', '1250', 'br', "it's", 'does',
              'doesn', 'don', 'mwork', '2.2']
for _ in text.ENGLISH_STOP_WORDS:
    if _ in keep_words:
        pass
    else:
        review_stop_words.append(_)
for _ in test_stop_words:
    review_stop_words.append(_)
    
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()    
    
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def vectorize_this(df, max_features, min_df, max_df, ngram_max):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             strip_accents = 'unicode',
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(1, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    return X, vectorizer

In [8]:
topics1 = 5
mindf1 = 2
maxdf1 = .95
grams1 = 2

In [11]:
X1, vectorizer1 = vectorize_this(df1, 3000, mindf1, maxdf1, grams1)

In [96]:
def make_nmf(n_components, alpha, X):
    nmf = NMF(
            n_components=n_components,
            init='nndsvda',
            solver='mu',
            beta_loss='kullback-leibler', 
            random_state=12345,
            alpha = alpha
            )

    W = nmf.fit_transform(X)
    H = nmf.components_
    return nmf, W, H

In [97]:
nmf1, W1, H1, = make_nmf(topics1, .1, X1)

In [100]:
def topic_keywords(nmf_model, vectorizer, n_words=10):

    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in nmf_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [101]:
list1 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[4]).argsort()[:500]).tolist()
list1[:5]

['sensor', 'food', 'cook', 'reheat', 'defrost']

In [102]:
list2=df1.model_ready[368].split()
list2[:5]

['genius', 'wave', 'lasted', 'compared', 'convection']

In [103]:
nmf1.reconstruction_err_

241.62213562286385

In [104]:
((-nmf1.components_[]).argsort()[:10])

SyntaxError: invalid syntax (<ipython-input-104-1a3df736b41b>, line 1)

In [ ]:
len(W1), len(df1)

In [ ]:
for num in W1[:,4].argsort()[-5:]:
    print(df1.review_body[num])
    print(df1.model_ready[num])
    print(review_df1.loc[(num)])

In [105]:
num = 149


In [106]:
toplist_1 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[0]).argsort()[:500]).tolist()
toplist_2 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[1]).argsort()[:500]).tolist()
toplist_3 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[2]).argsort()[:500]).tolist()
toplist_4 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[3]).argsort()[:500]).tolist()
toplist_5 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[4]).argsort()[:500]).tolist()
# toplist_6 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[5]).argsort()[:500]).tolist()
# toplist_7 = np.array(vectorizer1.get_feature_names()).take((-nmf1.components_[6]).argsort()[:500]).tolist()

sample=df1.model_ready[368].split()

sam_v_1 = []
sam_v_2 = []
sam_v_3 = []
sam_v_4 = []
sam_v_5 = []
sam_v_6 = []
sam_v_7 = []

for idx, i in enumerate(toplist_1):
    if i in sample:
#         sam_v_1.append(str(i) + " : " + str(idx))
        sam_v_1.append(idx)
for idx, i in enumerate(toplist_2):
    if i in sample:
#         sam_v_2.append(str(i) + " : " + str(idx))
        sam_v_2.append(idx)

for idx, i in enumerate(toplist_3):
    if i in sample:
#         sam_v_3.append(str(i) + " : " + str(idx))
        sam_v_3.append(idx)
for idx, i in enumerate(toplist_4):
    if i in sample:
#         sam_v_4.append(str(i) + " : " + str(idx))
        sam_v_4.append(idx)
for idx, i in enumerate(toplist_5):
    if i in sample:
#         sam_v_5.append(str(i) + " : " + str(idx))
        sam_v_5.append(idx)
# for idx, i in enumerate(toplist_6):
#     if i in sample:
# #         sam_v_6.append(str(i) + " : " + str(idx))
#         sam_v_6.append(idx)
# for idx, i in enumerate(toplist_7):
#     if i in sample:
# #         sam_v_7.append(str(i) + " : " + str(idx))
#         sam_v_7.append(idx)

In [107]:
df1.review_body[368], review_df1.loc[(368)]

('I had a Panasonic Genius as my first micro wave oven and it lasted over 20 years compared to a Convection Sharp that need to be<br />repaired by the 2nd year and 3 times after that.  I have only had it a few weeks, but I already love it.  Would recommend it to everyone.  I am also very impressed with Amazon.com.  They do a<br />tremendous job in getting out the orders and exactly when they say they will deliver.  I am very<br />impressed.',
 review_body                 I had a Panasonic Genius as my first micro wav...
 Size                                                                  0.99861
 Door & Other Breaks                                                     5e-05
 Lifetime                                                              0.00067
 Power / Power Settings                                                0.00027
 Pre-Programmed Functions                                              0.00041
 Name: 368, dtype: object)

In [108]:
len(sam_v_1[:]), len(sam_v_2[:]), len(sam_v_3[:]), len(sam_v_4[:]), len(sam_v_5[:]), len(sam_v_6[:]), len(sam_v_7[:])

(8, 7, 11, 5, 7, 0, 0)

In [109]:
sum(sam_v_1[:]), sum(sam_v_2[:]), sum(sam_v_3[:]), sum(sam_v_4[:]), sum(sam_v_5[:]), sum(sam_v_6[:]), sum(sam_v_7[:])

(1081, 937, 1347, 597, 1182, 0, 0)

In [110]:
statistics.mean(sam_v_1[:]), statistics.mean(sam_v_2[:]), statistics.mean(sam_v_3[:]), statistics.mean(sam_v_4[:]), statistics.mean(sam_v_5[:]), statistics.mean(sam_v_6[:]), statistics.mean(sam_v_7[:])

StatisticsError: mean requires at least one data point

In [111]:
len_diff(sam_v_1[:]), len_diff(sam_v_2[:]), len_diff(sam_v_3[:]), len_diff(sam_v_4[:]), len_diff(sam_v_5[:]), len_diff(sam_v_6[:]), len_diff(sam_v_7[:])

ZeroDivisionError: division by zero

In [112]:
def len_diff(list_a):
    diffs = []
    for i, e in enumerate(list_a):
        for j, f in enumerate(list_a):
            if i != j: diffs.append(abs(e-f))

    return sum(diffs)/len(diffs)



In [113]:
topic_keywords1 = topic_keywords(nmf1, vectorizer1)        

In [114]:
topic_keywords1

[array(['large', 'big', 'fit', 'space', 'size', 'kitchen', 'powerful',
        'larger', 'need', 'counter'], dtype='<U21'),
 array(['door', 'open', 'close', 'latch', 'month', 'button', 'problem',
        'broke', 'push', 'door latch'], dtype='<U21'),
 array(['model', 'new', 'lasted', 'died', 'service', 'month', 'purchased',
        'price', 'working', 'replace'], dtype='<U21'),
 array(['power', 'time', 'popcorn', 'watt', 'setting', 'level', 'button',
        'technology', 'minute', 'second'], dtype='<U21'),
 array(['sensor', 'food', 'cook', 'reheat', 'defrost', 'microwav', 'heat',
        'sensor reheat', 'function', 'evenly'], dtype='<U21')]

In [115]:
W1.shape, H1.shape

((1854, 5), (5, 3000))

In [116]:
nmf1.components_.shape

(5, 3000)

In [117]:
topic_keywords1 = topic_keywords(nmf1, vectorizer1)        

In [118]:
# topic_keywords2 = topic_keywords(vectorizer2, nmf2)        

In [119]:
# topic_keywords3 = topic_keywords(vectorizer3, nmf3)        

In [120]:
topic_keywords1

[array(['large', 'big', 'fit', 'space', 'size', 'kitchen', 'powerful',
        'larger', 'need', 'counter'], dtype='<U21'),
 array(['door', 'open', 'close', 'latch', 'month', 'button', 'problem',
        'broke', 'push', 'door latch'], dtype='<U21'),
 array(['model', 'new', 'lasted', 'died', 'service', 'month', 'purchased',
        'price', 'working', 'replace'], dtype='<U21'),
 array(['power', 'time', 'popcorn', 'watt', 'setting', 'level', 'button',
        'technology', 'minute', 'second'], dtype='<U21'),
 array(['sensor', 'food', 'cook', 'reheat', 'defrost', 'microwav', 'heat',
        'sensor reheat', 'function', 'evenly'], dtype='<U21')]

In [121]:
# def topic_featuring(X, topic_keywords, n_components=10, n_words=10):
    
#     nmf, W, H = make_nmf(n_components, .1, X)
    
#     # Topic - Keywords Dataframe
#     df_topic_keywords = pd.DataFrame(topic_keywords)
#     df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
#     df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

#     Topics_theme = range(n_components)
#     df_topic_keywords['topic_theme'] = Topics_theme
#     df_topic_keywords.set_index('topic_theme', inplace=True)
#     return df_topic_keywords.T

In [122]:
# topic_featuring(X1, topic_keywords1, topics1)

In [123]:
# topic_featuring(X2, topic_keywords2, topics2)

In [124]:
# topic_featuring(X3, topic_keywords3, topics3)

In [125]:
def df_featurizer(df, topic_keywords, topic_labels=None, n_components=10, column_names_known = 'n'):

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    
    if column_names_known == 'n':
        Topics_theme = range(n_components)
    elif column_names_known == 'Y':
        Topics_theme = topic_labels
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [126]:
df_featurizer(df, topic_keywords1, n_components=topics1)

topic_theme,0,1,2,3,4
Word 0,large,door,model,power,sensor
Word 1,big,open,new,time,food
Word 2,fit,close,lasted,popcorn,cook
Word 3,space,latch,died,watt,reheat
Word 4,size,month,service,setting,defrost
Word 5,kitchen,button,month,level,microwav
Word 6,powerful,problem,purchased,button,heat
Word 7,larger,broke,price,technology,sensor reheat
Word 8,need,push,working,minute,function
Word 9,counter,door latch,replace,second,evenly


In [127]:
# nmf_featurizer(df2, max_df=maxdf2, ngram_max=grams2, n_components=topics2)

In [128]:
# nmf_featurizer(df3, max_df=maxdf3, ngram_max=grams3, n_components=topics3)

In [129]:
vocabulary1 = np.array(vectorizer1.get_feature_names())
vocabulary1

array(['0', '00', '000', ..., 'youtube', 'zap', 'zero'], dtype='<U21')

In [130]:
# vocabulary2 = np.array(vectorizer2.get_feature_names())

In [131]:
# vocabulary3 = np.array(vectorizer3.get_feature_names())

In [132]:
def label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    topic_labels = []
    for i, row in enumerate(H):
        top_words = np.argsort(row)[::-1][:12]
        print('topic', i)
        print('-->', ' '.join(vocabulary[top_words]))
        label = str(input('please label this topic: '))
        topic_labels.append(label)
        print()
    return topic_labels

In [133]:
topic_labels1 = label_topics(H1,vocabulary1)

topic 0
--> large big fit space size kitchen powerful larger need counter cooking cook
please label this topic: Size

topic 1
--> door open close latch month button problem broke push door latch hard working
please label this topic: Door & Other Breaks

topic 2
--> model new lasted died service month purchased price working replace replaced brand
please label this topic: Lifetime

topic 3
--> power time popcorn watt setting level button technology minute second inverter cook
please label this topic: Power / Power Settings

topic 4
--> sensor food cook reheat defrost microwav heat sensor reheat function evenly month button
please label this topic: Pre-Programmed Functions



In [134]:
# topic_labels2 = label_topics(H2,vocabulary2)

In [135]:
# topic_labels3 = label_topics(H3,vocabulary3)

In [136]:
df_featurizer(df, topic_keywords1, topic_labels1, n_components=topics1, column_names_known = 'Y')

topic_theme,Size,Door & Other Breaks,Lifetime,Power / Power Settings,Pre-Programmed Functions
Word 0,large,door,model,power,sensor
Word 1,big,open,new,time,food
Word 2,fit,close,lasted,popcorn,cook
Word 3,space,latch,died,watt,reheat
Word 4,size,month,service,setting,defrost
Word 5,kitchen,button,month,level,microwav
Word 6,powerful,problem,purchased,button,heat
Word 7,larger,broke,price,technology,sensor reheat
Word 8,need,push,working,minute,function
Word 9,counter,door latch,replace,second,evenly


In [137]:
# df_featurizer(df2, topic_keywords2, topic_labels2, n_components=topics2, column_names_known = 'Y')

In [138]:
# df_featurizer(df3, topic_keywords3, topic_labels3, n_components=topics3, column_names_known = 'Y')

In [139]:
def softmax(v, temperature=.01):
    '''
    A heuristic to convert arbitrary positive values into probabilities.
    See: https://en.wikipedia.org/wiki/Softmax_function
    '''
    expv = np.exp(v / temperature)
    s = np.sum(expv)
    return expv / s

In [140]:
def analyze_reviews(W, topic_labels):
    '''
    Print an analysis of a single Amazon review, including the review ID
    and a summary of which topics it represents. The topics are identified
    via the hand-labels which were assigned by the user.
    '''
    topic_count = len(topic_labels)
    topic_percentages = [[] for i in range(0, topic_count)]

    for idx, i in enumerate(W):  
        probs = softmax(W[idx], temperature=.01)
        topic_list_counter = 0
        for prob, label in zip(probs, topic_labels):
            topic_percentages[topic_list_counter].append(round(prob, 5))
            topic_list_counter += 1
        
    return pd.DataFrame(dict(zip(topic_labels, topic_percentages)))

In [141]:
top_df1 = analyze_reviews(W1, topic_labels1)

In [142]:
# top_df2 = analyze_reviews(W2, topic_labels2)

In [143]:
# top_df3 = analyze_reviews(W3, topic_labels3)

In [144]:
top_df1

,Size,Door & Other Breaks,Lifetime,Power / Power Settings,Pre-Programmed Functions
0,0.00000,0.99999,0.00000,0.00000,0.00000
1,0.99822,0.00044,0.00044,0.00044,0.00044
2,0.57098,0.04851,0.04851,0.04851,0.28347
3,0.01092,0.01092,0.95633,0.01092,0.01092
4,0.00000,0.99993,0.00000,0.00000,0.00006
...,...,...,...,...,...
1849,0.99952,0.00009,0.00009,0.00009,0.00022
1850,0.00122,0.00028,0.00028,0.00028,0.99794
1851,0.05473,0.00775,0.00775,0.76654,0.16323
1852,0.05803,0.01090,0.01090,0.90927,0.01090


In [145]:
top_df1.loc[0]

Size                        0.00000
Door & Other Breaks         0.99999
Lifetime                    0.00000
Power / Power Settings      0.00000
Pre-Programmed Functions    0.00000
Name: 0, dtype: float64

In [146]:
def feature_score(top_df):    
    feature_score = {}
    for _ in range(len(top_df.columns)):
        key = top_df.columns[_]
        feature_score[key] = round(sum(top_df[key])/(len(top_df)),5)
    return feature_score

In [147]:
feature_score1 = feature_score(top_df1)
sorted(feature_score1.items(), key= lambda x: x[1], reverse=True)

[('Size', 0.25259),
 ('Lifetime', 0.23249),
 ('Door & Other Breaks', 0.1819),
 ('Power / Power Settings', 0.16816),
 ('Pre-Programmed Functions', 0.16485)]

In [148]:
# feature_score2 = feature_score(top_df2)
# sorted(feature_score2.items(), key= lambda x: x[1], reverse=True)

In [149]:
# feature_score3 = feature_score(top_df3)
# sorted(feature_score3.items(), key= lambda x: x[1], reverse=True)

In [150]:
topic_words_df1 = df_featurizer(df1, topic_keywords1, topic_labels1, column_names_known ='Y')
# topic_words_df2 = nmf_featurizer(df2, topic_keywords2, topic_labels2, column_names_known ='Y')
# topic_words_df3 = nmf_featurizer(df3, topic_keywords3, topic_labels3, column_names_known ='Y')


In [151]:
topic_words_df1

topic_theme,Size,Door & Other Breaks,Lifetime,Power / Power Settings,Pre-Programmed Functions
Word 0,large,door,model,power,sensor
Word 1,big,open,new,time,food
Word 2,fit,close,lasted,popcorn,cook
Word 3,space,latch,died,watt,reheat
Word 4,size,month,service,setting,defrost
Word 5,kitchen,button,month,level,microwav
Word 6,powerful,problem,purchased,button,heat
Word 7,larger,broke,price,technology,sensor reheat
Word 8,need,push,working,minute,function
Word 9,counter,door latch,replace,second,evenly


In [152]:
review_df_columns1 = (['review_body'] + topic_labels1)
# review_df_columns2 = (['review_body'] + topic_labels2)
# review_df_columns3 = (['review_body'] + topic_labels3)

In [153]:
review_df1 = pd.concat([df1, top_df1.reindex(df1.index)], axis=1)[review_df_columns1]
# review_df2 = pd.concat([df2, top_df2.reindex(df2.index)], axis=1)[review_df_columns2]
# review_df3 = pd.concat([df3, top_df3.reindex(df3.index)], axis=1)[review_df_columns3]

In [154]:
def review_checker(review_df, num_samples):
    review_df = review_df
    review_index = []
    for _ in range(num_samples):
        review_index.append(random.randint(1, len(review_df)))
        
    for _ in review_index:
        print('Review ' + str(_))
        print(' ')
        print(review_df.review_body[_])
        print(' ')
        print(review_df.loc[_][1:])
        print(' ')
        print('___________________________________________________')
        print(' ')
    

In [155]:
len(W1)

1854

In [156]:
W1[:,0].argsort()[-1:]

array([1244])

In [157]:
H1[:,0].argsort()[:]

array([2, 4, 1, 0, 3])

In [158]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4471, 4472, 4473, 4474, 4475, 4476, 4477, 4478, 4479, 4480],
           dtype='int64', length=4356)

In [159]:
np.array(vectorizer1.get_feature_names()).take(nmf1.components_[1].argsort()[:])[-100:]

array(['office', 'going', 'march', 'press', 'model', 'properly', 'money',
       'sound', 'pushing', 'failed', 'problem door', 'try', 'purchase',
       '2014', 'disappointed', 'read', 'feel', 'purchased', 'door hard',
       'closing door', 'close door', 'opened', 'machine', 'open close',
       'pull', '2013', 'piece junk', 'stick', 'hard close', 'quality',
       'second', 'later', 'buying', 'button open', 'little', 'cheap',
       'make', 'completely', 'poor', 'think', 'issue', 'break', 'waste',
       'star', 'design', 'noise', 'lasted', 'release', 'hand', 'half',
       'slam door', 'piece', 'longer', 'stop', 'ago', 'loud', 'warranty',
       'started', 'stay', 'mechanism', 'closed', 'switch', 'junk',
       'closing', 'complaint', 'push button', 'repair', 'stopped working',
       'worked fine', 'shop', 'fix', 'plastic', 'door close', 'turn',
       'broken', 'week', 'off', 'light', 'slam', 'opening', 'stopped',
       'open door', 'time', 'door open', 'fine', 'start', 'worked',

In [160]:

for idx, i in enumerate(W1[:,2].argsort()[:]):
    if i == 368:
        print(idx)

848


In [161]:
df

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,model_ready
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-27,91,I have had this microwave for just over 3 year...,door close know door closed turn irritating ha...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-26,23,Man this thing cooks evenly and fast! It takes...,man cook evenly fast bit room counter room spa...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-23,17,Works great! Have had it over a year and no su...,surprise large microwav
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-23,13,I ordered and received this several years ago ...,ordered received ago
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-22,131,I have had this microwave for just over 2 year...,door close know door closed turn google proble...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476,16110779,R3FC9T1YVGUB5V,B005BFZ5N6,4.0,24.0,28.0,Y,Nice unit so far..,My review here is pretty simple as we've only ...,NaT,350,My review here is pretty simple as we've only ...,pretty simple sd681s week regret new quite res...
4477,52890284,R3TNQ6FQVJ5FBU,B005BFZ5N6,5.0,185.0,197.0,Y,Happy with it,"Another reviewer gave a lot more details, and ...",NaT,216,"Another reviewer gave a lot more details, and ...",reviewer gave helpful I short stick noticed li...
4478,52894480,R2VKU64KO1E3VP,B005BFZ5N6,5.0,110.0,122.0,Y,A modest improvement over its predecessor,"In most ways, this microwave oven is similar t...",NaT,982,"In most ways, this microwave oven is similar t...",similar earlier asin b001v7r5i6 prestige sd688...
4479,51028962,RP9CT6IK8VNCV,B005BFZ5N6,5.0,15.0,17.0,Y,Very nice so far,This is a pretty good looking microwave and a ...,NaT,142,This is a pretty good looking microwave and a ...,pretty looking model listed place sensor rehea...


In [162]:
df1.model_ready[368]

'genius wave lasted compared convection need berepaired 2nd time week impressed atremendous job getting order exactly deliver veryimpress'

In [163]:
df1.model_ready[368], review_df1.loc[(368)]

('genius wave lasted compared convection need berepaired 2nd time week impressed atremendous job getting order exactly deliver veryimpress',
 review_body                 I had a Panasonic Genius as my first micro wav...
 Size                                                                  0.99978
 Door & Other Breaks                                                     1e-05
 Lifetime                                                                1e-05
 Power / Power Settings                                                  1e-05
 Pre-Programmed Functions                                              0.00019
 Name: 368, dtype: object)

In [164]:
review_checker(review_df1, 50)

Review 364
 
Purchased this microwave exactly a year ago.  It worked OK for the most part except the door was very stiff to close.  I believe that this issue with the door is what cause it to stop working.<br /><br />Would not recommend it or if you buy it and notice the trouble with the door, return it right way as it will eventually be the cause of its demise.
 
Size                        0.00818
Door & Other Breaks         0.98779
Lifetime                    0.00134
Power / Power Settings      0.00134
Pre-Programmed Functions    0.00134
Name: 364, dtype: object
 
___________________________________________________
 
Review 324
 
This is a perfect little microwave.  Compact but powerful, and I can turn off ALL of the sound so I don't have to hear it across the house or late/early when half the house is asleep.  Well worth the price.  Great buy.
 
Size                        0.99119
Door & Other Breaks          0.0022
Lifetime                     0.0022
Power / Power Settings       0